In [1]:
import numpy as np 
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from IPython.display import Image


import statsmodels.api as sma

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso,Ridge,ElasticNet,SGDRegressor
import statsmodels.stats.api as ssa
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
#from mlxtend.feature_selection import SequentialFeatureSelector

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

In [2]:
df=pd.read_csv("media prediction and its cost.csv")
df.head()

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,total_children,education,member_card,occupation,houseowner,avg_cars_at home(approx),avg. yearly_income,num_children_at_home,avg_cars_at home(approx).1,brand_name,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,store_type,store_city,store_state,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4.0,Bag Stuffers,USA,M,F,1.0,Partial High School,Normal,Skilled Manual,Y,1.0,$10K - $30K,1.0,1.0,Carrington,1.84,19.70,17.70,1.0,0.0,17.0,Deluxe Supermarket,Salem,OR,27694.0,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3.0,Cash Register Lottery,USA,M,M,0.0,Bachelors Degree,Silver,Professional,Y,4.0,$50K - $70K,0.0,4.0,Carrington,1.84,19.70,17.70,1.0,0.0,17.0,Deluxe Supermarket,Salem,OR,27694.0,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",59.86
2,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2.0,High Roller Savings,USA,S,F,4.0,Partial High School,Normal,Manual,N,1.0,$10K - $30K,0.0,1.0,Carrington,1.84,19.70,17.70,1.0,0.0,17.0,Deluxe Supermarket,Salem,OR,27694.0,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",84.16
3,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2.0,Cash Register Lottery,USA,M,F,2.0,High School Degree,Bronze,Manual,Y,2.0,$30K - $50K,2.0,2.0,Carrington,1.84,19.70,17.70,1.0,0.0,17.0,Deluxe Supermarket,Salem,OR,27694.0,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,In-Store Coupon,95.78
4,Breakfast Foods,Frozen Foods,Food,4.08,1.4280,3.0,Double Down Sale,USA,M,M,0.0,Partial High School,Bronze,Skilled Manual,N,2.0,$30K - $50K,0.0,2.0,Golden,1.36,7.12,5.11,0.0,1.0,29.0,Deluxe Supermarket,Salem,OR,27694.0,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,Radio,50.79


In [3]:
num_df = df.select_dtypes(include=np.number)
cat_df = df.select_dtypes(exclude=np.number)
df = df.drop('avg_cars_at home(approx)',axis=1)
num_df_to_be_power_transformed = num_df.drop(['total_children','avg_cars_at home(approx)','avg_cars_at home(approx).1','num_children_at_home','recyclable_package', 'low_fat', 'coffee_bar', 'video_store','salad_bar', 'prepared_food', 'florist'],axis=1)

In [4]:
#label encoding for martial status,gender and houseowner
label_encoder = LabelEncoder()
df['marital_status'] = label_encoder.fit_transform(df['marital_status'])
df['gender'] = label_encoder.fit_transform(df['gender'])
df['houseowner'] = label_encoder.fit_transform(df['houseowner'])



# Ordinal encoding for 'sales_country'
df['sales_country'] = df['sales_country'].replace('USA',2).replace('Mexico',1).replace('Canada',0)

# Ordinal encoding for 'member_card'
member_card_order = {'Golden': 1, 'Silver': 0, 'Bronze': 3, 'Normal': 2}
df['member_card'] = df['member_card'].map(member_card_order)

# Ordinal encoding for 'education'
education_order = {'Partial High School': 4, 'Bachelors Degree': 2, 'High School Degree': 3,
                       'Graduate Degree': 0, 'Partial College': 1}
df['education'] = df['education'].map(education_order)
# Ordinal encoding for 'store_type'
store_type_order = {'Deluxe Supermarket': 3, 'Supermarket': 4, 'Gourmet Supermarket': 2,
                        'Small Grocery': 0, 'Mid-Size Grocery': 1}
df['store_type'] = df['store_type'].map(store_type_order)
    
    
# Ordinal encoding for 'avg. yearly_income'
income_order = {
        '$10K - $30K': 6,
        '$30K - $50K': 7,
        '$50K - $70K': 5,
        '$70K - $90K': 3,
        '$90K - $110K': 2,
        '$110K - $130K': 4,
        '$130K - $150K': 1,
        '$150K +': 0
}

df['avg. yearly_income'] = df['avg. yearly_income'].map(income_order)


In [6]:
multiple_values_columns = ['occupation','food_category','food_department','promotion_name','brand_name','store_city','store_state','media_type']

for col in multiple_values_columns:
        melt_with_cost = pd.melt(df,id_vars=col,value_vars='cost',value_name='cost')
        mean_cost = np.ceil(melt_with_cost.groupby(by=col)['cost'].mean()).to_dict()
        df[col] = df[col].map(mean_cost)

In [7]:
df['food_family'] = df['food_family'].replace('Food',2).replace('Non-Consumable',1).replace('Drink',0)

In [8]:
num_df_to_be_power_transformed = num_df_to_be_power_transformed.drop('cost',axis=1)

In [9]:
df1 = df.copy()

In [10]:
for i in num_df_to_be_power_transformed.columns:
    df1[i] = df[i].apply(lambda x:np.sqrt(x))

In [11]:
df1.head()

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,total_children,education,member_card,occupation,houseowner,avg. yearly_income,num_children_at_home,avg_cars_at home(approx).1,brand_name,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,store_type,store_city,store_state,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,100.0,100.0,2,2.712932,1.650212,2.000000,107.0,2,0,0,1.0,4,2,100.0,1,6,1.0,1.0,100.0,1.356466,4.438468,4.207137,1.0,0.0,4.123106,3,85.0,95.0,166.415144,136.638208,73.586684,60.083276,1.0,1.0,1.0,1.0,1.0,98.0,126.62
1,100.0,100.0,2,2.349468,1.610714,1.732051,75.0,2,0,1,0.0,2,0,99.0,1,5,0.0,4.0,100.0,1.356466,4.438468,4.207137,1.0,0.0,4.123106,3,85.0,95.0,166.415144,136.638208,73.586684,60.083276,1.0,1.0,1.0,1.0,1.0,98.0,59.86
2,100.0,100.0,2,1.918333,1.166876,1.414214,96.0,2,1,0,4.0,4,2,99.0,0,6,0.0,1.0,100.0,1.356466,4.438468,4.207137,1.0,0.0,4.123106,3,85.0,95.0,166.415144,136.638208,73.586684,60.083276,1.0,1.0,1.0,1.0,1.0,98.0,84.16
3,100.0,100.0,2,1.918333,1.085173,1.414214,75.0,2,0,0,2.0,3,3,99.0,1,7,2.0,2.0,100.0,1.356466,4.438468,4.207137,1.0,0.0,4.123106,3,85.0,95.0,166.415144,136.638208,73.586684,60.083276,1.0,1.0,1.0,1.0,1.0,100.0,95.78
4,100.0,100.0,2,2.019901,1.194990,1.732051,81.0,2,0,1,0.0,4,3,100.0,0,7,0.0,2.0,101.0,1.166190,2.668333,2.260531,0.0,1.0,5.385165,3,85.0,95.0,166.415144,136.638208,73.586684,60.083276,1.0,1.0,1.0,1.0,1.0,97.0,50.79


In [ ]:
X1 = df1.drop(columns='cost')
y1 = df1['cost']

X_train1,X_test1,y_train1,y_test1 = train_test_split(X1,y1,test_size=.20,random_state=1)

In [ ]:
import statsmodels.api as sma
df_constant = sma.add_constant(X_train1)
model=sma.OLS(y_train1,df_constant)
result=model.fit()
print(result.summary())

In [ ]:
lr = LinearRegression()

base_model_sqrt = lr.fit(X_train1,y_train1)

train_pred_sqrt = base_model_sqrt.predict(X_train1)
test_pred_sqrt = base_model_sqrt.predict(X_test1)
r2_train_sqrt = r2_score(y_train1,train_pred_sqrt)
r2_test_sqrt = r2_score(y_test1,test_pred_sqrt)
print('R2 Train:',r2_train_sqrt)
print('R2 Test:',r2_test_sqrt)
print('RMSE Train:',np.sqrt(mean_squared_error(y_train1,train_pred_sqrt)))
print('RMSE test:',np.sqrt(mean_squared_error(y_test1,test_pred_sqrt)))

In [ ]:
def backward_elimination_vif(data_frame, vif_threshold=10):
    selected_cols = data_frame.columns.tolist()
    
    while len(selected_cols) > 0:
        vif_values = [variance_inflation_factor(data_frame[selected_cols].values, i) for i in range(len(selected_cols))]
        
        max_vif_index = vif_values.index(max(vif_values))
        max_vif = max(vif_values)
        
        if max_vif < vif_threshold:
            break
        
        selected_cols_before_remove = selected_cols.copy()
        
        removed_col = selected_cols.pop(max_vif_index)
        
        if len(selected_cols) == 0:
            print(f"Error occurred at VIF threshold: {vif_threshold}")
            return data_frame[selected_cols_before_remove]
        
        print(f"Removed '{removed_col}' (VIF: {max_vif:.2f})")
    
    selected_data = data_frame[selected_cols]
    
    return selected_data

In [ ]:
selected_data = backward_elimination_vif(X_train1, vif_threshold=10)

In [ ]:
selected_vif_columns = selected_data.columns.to_list()
selected_vif_columns

In [ ]:
X_train1_vif = X_train1[['food_family',
 'sales_country',
 'marital_status',
 'gender',
 'total_children',
 'education',
 'member_card',
 'houseowner',
 'avg. yearly_income',
 'num_children_at_home',
 'avg_cars_at home(approx).1',
 'recyclable_package',
 'low_fat',
 'units_per_case',
 'coffee_bar',
 'video_store',
 'prepared_food',
 'florist']]

In [ ]:
X_test1_vif  = X_test1[['food_family',
 'sales_country',
 'marital_status',
 'gender',
 'total_children',
 'education',
 'member_card',
 'houseowner',
 'avg. yearly_income',
 'num_children_at_home',
 'avg_cars_at home(approx).1',
 'recyclable_package',
 'low_fat',
 'units_per_case',
 'coffee_bar',
 'video_store',
 'prepared_food',
 'florist']]

In [ ]:
lr = LinearRegression()
model4 = lr.fit(X_train1_vif,y_train1)
pred_train_4 =  model4.predict(X_train1_vif)
pred_test_4 = model4.predict(X_test1_vif)
r2_train_4 = r2_score(y_train1,pred_train_4)
r2_test_4 = r2_score(y_test1,pred_test_4)
print('R2 Train:',r2_train_4)
print('R2 Test:',r2_test_4)
print('RMSE Train:',np.sqrt(mean_squared_error(y_train1,pred_train_4)))
print('RMSE Test:',np.sqrt(mean_squared_error(y_test1,pred_test_4)))

In [ ]:
kf = KFold(n_splits=5,shuffle=True, random_state=1)


lr = LinearRegression()
prediction_ml = []
rmse_kf = []

for train_index, test_index in kf.split(X_train1):
    kf_xtrain = X_train1.iloc[train_index]
    kf_xtest = X_train1.iloc[test_index]
    kf_ytrain = y_train1.iloc[train_index]
    kf_ytest = y_train1.iloc[test_index]
    
    prediction_ml.append(lr.fit(kf_xtrain, kf_ytrain).predict(kf_xtest))
    
    
    kfold=lr.fit(kf_xtrain,kf_ytrain)
    pred = kfold.predict(kf_xtest)
    rmse_kf.append(np.sqrt(mean_squared_error(kf_ytest,pred)))

print(rmse_kf)

In [ ]:
np.mean(rmse_kf)

In [ ]:
lr = LinearRegression()

loocv = LeaveOneOut()

loocv_rmse = []
loocv_score = []


for train_index, test_index in loocv.split(X_train1):
    loocv_xtrain = X_train1.iloc[train_index]
    loocv_xtest = X_train1.iloc[test_index]
    loocv_ytrain = y_train1.iloc[train_index]
    loocv_ytest = y_train1.iloc[test_index]
    
    loocv_score.append(lr.fit(loocv_xtrain, loocv_ytrain).predict(loocv_xtest))
    
    
    loocv=lr.fit(loocv_xtrain,loocv_ytrain)
    pred = loocv.predict(loocv_xtest)
    loocv_rmse.append(np.sqrt(mean_squared_error(loocv_ytest,pred)))
    
print(loocv_rmse)